# Notebook for Segmentation and Clustering Neighborhoods in Toronto


In [1]:
import pandas as pd #For Data analysis
import numpy as np

import requests # For URL accessibilty

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Get the Wiki page data

In [2]:
wiki_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

web_data=requests.get(wiki_url).text
print("Data obtained from Wiki")

Data obtained from Wiki


### Use Beautiful Soap to extract the data

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(web_data,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptJpgpAICsAAFpdq2oAAAAK","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

#### Find the table to get the data

In [4]:
PO_table = soup.find('table',{'class':'wikitable sortable'})
PO_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

#### Use the tags to seperate the relevant data and make a list

The tags are as below

<b>*th*</b> for the column headers.
<b>*td*</b> for the column values.

In [5]:
columns_tag = PO_table.findAll('th')
columns_tag

[<th>Postal code
 </th>,
 <th>Borough
 </th>,
 <th>Neighborhood
 </th>]

In [6]:
import re

clean = re.compile('<.*?>')
columns = []
for tag in columns_tag:
    columns.append(re.sub(clean,"", str(tag)).strip('\n'))

columns


['Postal code', 'Borough', 'Neighborhood']

In [7]:
values_tag = PO_table.findAll('td')
values_tag

[<td>M1A
 </td>,
 <td>Not assigned
 </td>,
 <td>
 </td>,
 <td>M2A
 </td>,
 <td>Not assigned
 </td>,
 <td>
 </td>,
 <td>M3A
 </td>,
 <td>North York
 </td>,
 <td>Parkwoods
 </td>,
 <td>M4A
 </td>,
 <td>North York
 </td>,
 <td>Victoria Village
 </td>,
 <td>M5A
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Regent Park / Harbourfront
 </td>,
 <td>M6A
 </td>,
 <td>North York
 </td>,
 <td>Lawrence Manor / Lawrence Heights
 </td>,
 <td>M7A
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Queen's Park / Ontario Provincial Government
 </td>,
 <td>M8A
 </td>,
 <td>Not assigned
 </td>,
 <td>
 </td>,
 <td>M9A
 </td>,
 <td>Etobicoke
 </td>,
 <td>Islington Avenue
 </td>,
 <td>M1B
 </td>,
 <td>Scarborough
 </td>,
 <td>Malvern / Rouge
 </td>,
 <td>M2B
 </td>,
 <td>Not assigned
 </td>,
 <td>
 </td>,
 <td>M3B
 </td>,
 <td>North York
 </td>,
 <td>Don Mills
 </td>,
 <td>M4B
 </td>,
 <td>East York
 </td>,
 <td>Parkview Hill / Woodbine Gardens
 </td>,
 <td>M5B
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Garden Distric

In [8]:
values =[]

for tag in values_tag:
    values.append(re.sub(clean,"",str(tag)).strip('\n'))

values

['M1A',
 'Not assigned',
 '',
 'M2A',
 'Not assigned',
 '',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Regent Park / Harbourfront',
 'M6A',
 'North York',
 'Lawrence Manor / Lawrence Heights',
 'M7A',
 'Downtown Toronto',
 "Queen's Park / Ontario Provincial Government",
 'M8A',
 'Not assigned',
 '',
 'M9A',
 'Etobicoke',
 'Islington Avenue',
 'M1B',
 'Scarborough',
 'Malvern / Rouge',
 'M2B',
 'Not assigned',
 '',
 'M3B',
 'North York',
 'Don Mills',
 'M4B',
 'East York',
 'Parkview Hill / Woodbine Gardens',
 'M5B',
 'Downtown Toronto',
 'Garden District, Ryerson',
 'M6B',
 'North York',
 'Glencairn',
 'M7B',
 'Not assigned',
 '',
 'M8B',
 'Not assigned',
 '',
 'M9B',
 'Etobicoke',
 'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale',
 'M1C',
 'Scarborough',
 'Rouge Hill / Port Union / Highland Creek',
 'M2C',
 'Not assigned',
 '',
 'M3C',
 'North York',
 'Don Mills',
 'M4C',
 'East York',
 'W

#### Make a dictionary to later convert into a Data Frame

In [9]:
postal_code=[]
borough =[]
neighbourhoods=[]
i=0
while i<len(values):
    postal_code.append(values[i])
    borough.append(values[i+1])
    neighbourhoods.append(values[i+2])
    i+=3
    
data_for_table=[postal_code, borough, neighbourhoods]

final_data=dict(zip(columns, data_for_table))
final_data


{'Postal code': ['M1A',
  'M2A',
  'M3A',
  'M4A',
  'M5A',
  'M6A',
  'M7A',
  'M8A',
  'M9A',
  'M1B',
  'M2B',
  'M3B',
  'M4B',
  'M5B',
  'M6B',
  'M7B',
  'M8B',
  'M9B',
  'M1C',
  'M2C',
  'M3C',
  'M4C',
  'M5C',
  'M6C',
  'M7C',
  'M8C',
  'M9C',
  'M1E',
  'M2E',
  'M3E',
  'M4E',
  'M5E',
  'M6E',
  'M7E',
  'M8E',
  'M9E',
  'M1G',
  'M2G',
  'M3G',
  'M4G',
  'M5G',
  'M6G',
  'M7G',
  'M8G',
  'M9G',
  'M1H',
  'M2H',
  'M3H',
  'M4H',
  'M5H',
  'M6H',
  'M7H',
  'M8H',
  'M9H',
  'M1J',
  'M2J',
  'M3J',
  'M4J',
  'M5J',
  'M6J',
  'M7J',
  'M8J',
  'M9J',
  'M1K',
  'M2K',
  'M3K',
  'M4K',
  'M5K',
  'M6K',
  'M7K',
  'M8K',
  'M9K',
  'M1L',
  'M2L',
  'M3L',
  'M4L',
  'M5L',
  'M6L',
  'M7L',
  'M8L',
  'M9L',
  'M1M',
  'M2M',
  'M3M',
  'M4M',
  'M5M',
  'M6M',
  'M7M',
  'M8M',
  'M9M',
  'M1N',
  'M2N',
  'M3N',
  'M4N',
  'M5N',
  'M6N',
  'M7N',
  'M8N',
  'M9N',
  'M1P',
  'M2P',
  'M3P',
  'M4P',
  'M5P',
  'M6P',
  'M7P',
  'M8P',
  'M9P',
  'M1R',
  'M

In [57]:
df = pd.DataFrame(final_data)
df.rename(columns={'Postal code': 'Postal Code', 'Neighborhood': 'Neighborhoods'}, inplace=True)
df

,Postal Code,Borough,Neighborhoods
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


#### Drop the rows whose Borough is 'Not assigned'

In [58]:
df.drop(df[df['Borough']== 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


#### Make the Neighborhood column clean

In [59]:
df['Neighborhoods']=df['Neighborhoods'].str.replace('/', ',')
df

,Postal Code,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


#### Shape of the Data frame

In [60]:
df.shape

(103, 3)

### Add the latitude and longitude data to your Data frame

In [61]:
lat_long=pd.read_csv('Geospatial_Coordinates.csv')
lat_long

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [62]:
df= df.join(lat_long.set_index('Postal Code'), on='Postal Code')
df

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


### Explore Toronto Borough

In [63]:
Toronto_lat = 43.6532
Toronto_long = -79.3832

df_toronto = df[df['Borough'].str.endswith("Toronto")]
df_toronto

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259


#### Create a map of Toronto with neighborhoods superimposed on top.

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Toronto_lat, Toronto_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Write a function to get the nearby venues of Toronto Neighborhoods

In [86]:
# @hidden_cell
CLIENT_ID = 'HQCLNXBBJO1Q2OCKHDCNVHPNVUO45IQS334R3OWTY5FIKF2Y'
CLIENT_SECRET = 'IA2GDZO5HRQYYONOUDTNL3ATVPLNUEOZBT53STA040SMM45A'
VERSION = '20200428'
LIMIT = 100
radius = 500


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North &amp; West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Ro

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1635, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


#### Analyzing the neighbourhood

In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 237)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.015873,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.015873
5,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.013333,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026667
7,"Commerce Court , Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.040000,0.000000,0.00,...,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Top 10 venues per neighborhood

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Café,Bakery,Restaurant,Italian Restaurant,Seafood Restaurant,Farmers Market
1,"Brockton , Parkdale Village , Exhibition Place",Nightclub,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery,Pet Store,Music Venue,Climbing Gym,Restaurant
2,Business reply mail Processing CentrE,Light Rail Station,Recording Studio,Restaurant,Brewery,Butcher,Burrito Place,Auto Workshop,Spa,Fast Food Restaurant,Farmers Market
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Airport Terminal,Bar,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop,Thai Restaurant,Salad Place,Burger Joint,Fried Chicken Joint,Bubble Tea Shop


#### Lets cluster the Neighbourhoods

In [84]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

###### Lets add the top 10 venues to the cluster as well

In [69]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

toronto_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

###### Visualize the Clustering

In [85]:
# create map
map_clusters = folium.Map(location=[Toronto_lat, Toronto_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhoods'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### The Toronto Boroughs and its neighborhoods are clustered and visualized as above